In [44]:
import pandas as pd

df = pd.read_csv(r'C:\Users\Daniel\Documents\Trading\Trading Bot\EURUSD-H1-2Years.csv')
print(df.head())

  <DATE>\t<TIME>\t<OPEN>\t<HIGH>\t<LOW>\t<CLOSE>\t<TICKVOL>\t<VOL>\t<SPREAD>
0  2022.10.03\t00:00:00\t0.98008\t0.98008\t0.9783...                        
1  2022.10.03\t01:00:00\t0.97950\t0.98087\t0.9790...                        
2  2022.10.03\t02:00:00\t0.98003\t0.98028\t0.9789...                        
3  2022.10.03\t03:00:00\t0.97934\t0.98040\t0.9785...                        
4  2022.10.03\t04:00:00\t0.98015\t0.98286\t0.9801...                        


In [46]:
print(df.isnull().sum())
print(df.dtypes)
print(df.columns)

df.rename(columns={
    '<DATE>': 'Date',
    '<TIME>': 'Time',
    '<OPEN>': 'Open',
    '<HIGH>': 'High',
    '<LOW>': 'Low',
    '<CLOSE>': 'Close',
    '<TICKVOL>': 'TickVol',
    '<VOL>': 'Vol',
    '<SPREAD>': 'Spread'
}, inplace=True)

print(df.head())

df['Open'] = df['Open'].astype(float)
df['High'] = df['High'].astype(float)
df['Low'] = df['Low'].astype(float)
df['Close'] = df['Close'].astype(float)

<DATE>\t<TIME>\t<OPEN>\t<HIGH>\t<LOW>\t<CLOSE>\t<TICKVOL>\t<VOL>\t<SPREAD>    0
dtype: int64
<DATE>\t<TIME>\t<OPEN>\t<HIGH>\t<LOW>\t<CLOSE>\t<TICKVOL>\t<VOL>\t<SPREAD>    object
dtype: object
Index(['<DATE>\t<TIME>\t<OPEN>\t<HIGH>\t<LOW>\t<CLOSE>\t<TICKVOL>\t<VOL>\t<SPREAD>'], dtype='object')
  <DATE>\t<TIME>\t<OPEN>\t<HIGH>\t<LOW>\t<CLOSE>\t<TICKVOL>\t<VOL>\t<SPREAD>
0  2022.10.03\t00:00:00\t0.98008\t0.98008\t0.9783...                        
1  2022.10.03\t01:00:00\t0.97950\t0.98087\t0.9790...                        
2  2022.10.03\t02:00:00\t0.98003\t0.98028\t0.9789...                        
3  2022.10.03\t03:00:00\t0.97934\t0.98040\t0.9785...                        
4  2022.10.03\t04:00:00\t0.98015\t0.98286\t0.9801...                        


KeyError: 'Open'